# Calcul d'impact d'une pizza au thon

## Setup

Import des librairies de base

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import brightway2 as bw
from brightway2 import *
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from pprint import pprint

Setup du projet suite à l'exécution du script `importing_databases.py`

In [2]:
# list(bw.projects)
bw.projects.set_current("EF calculation")
bw.bw2setup()
agb = Database("agribalyse3")

Biosphere database already present!!! No setup is needed


Utilitaires

In [3]:
#climate_change = ('EF v3.0', 'climate change', 'global warming potential (GWP100)')
climate_change = ('EF v3.0 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')

def search_one(query):
    results = agb.search(query)
    if len(results) == 1:
        return results[0]
    else:
        filtered = list(filter(lambda activity: activity["name"] == query, results))
        assert len(filtered) == 1, f"search_one({query}) didn't return a single result: {results}"
        return results[0]

def get_demand(exchange, amount):
    activity = search_one(exchange["name"])

    return {activity: amount}

def list_impacts(activity, amount):
    for exchange in activity.technosphere():        
        updated_amount = exchange["amount"] * amount        
        demand = get_demand(exchange, updated_amount)
        lca = LCA(demand, climate_change)
        lca.lci()
        lca.lcia()
        print("*",updated_amount, exchange["unit"], exchange.input, "==>", lca.score)

## La pizza au thon

Sélection d'une "activité" à explorer : la pizza au thon

In [4]:
tuna_pizza_search = agb.search("tuna pizza")
print(f"{len(tuna_pizza_search)} results")
pprint(tuna_pizza_search)

5 results
['Pizza, tuna, at plant' (kilogram, FR, None),
 'Pizza, tuna, processed in FR | Chilled | Cardboard | at distribution/FR' (kilogram, None, None),
 'Pizza, tuna, processed in FR | Chilled | Cardboard | at packaging/FR' (kilogram, None, None),
 'Pizza, tuna, processed in FR | Chilled | Cardboard | at supermarket/FR' (kilogram, None, None),
 'Pizza, tuna, processed in FR | Chilled | Cardboard | Oven | at consumer/FR [Ciqual code: 26270]' (kilogram, None, None)]


Chaque produit passe par les 5 étapes suivante dans l'ordre :

- at plant (la recette)
- at packaging (conditionnement)
- at distribution (stockage)
- at supermarket (vente au détail)
- at consumer (consommation)

# Automation

In [ ]:
next_step = search_one("Pizza, tuna, at consumer")
amount = 1


while True:
    print(f">>>> {amount}kg", next_step["name"])
    demand = {next_step: amount}
    lca = LCA(demand, climate_change)
    lca.lci()
    lca.lcia()
    print("Total score:", lca.score)
    print()

    list_impacts(next_step, amount)
    
    input_products = list(filter(lambda exc: "Pizza, tuna" in exc["name"], next_step.technosphere()))
    if len(input_products) == 0:
        print("FINISHED")
        break
    if len(input_products) > 1:
        print("####### More than one product!", input_products)
        break
        
    next_step = input_products[0].input
    amount = input_products[0]["amount"] * amount
    print()


>>>> 1kg Pizza, tuna, processed in FR | Chilled | Cardboard | Oven | at consumer/FR [Ciqual code: 26270]
Total score: 2.633650099935815

* 1.0 kilogram 'Pizza, tuna, processed in FR | Chilled | Cardboard | at supermarket/FR' (kilogram, None, None) ==> 2.5661586639277627
* 0.0777 kilowatt hour 'Electricity, low voltage {FR}| market for | Cut-off, S - Copied from Ecoinvent' (kilowatt hour, None, None) ==> 0.004799317179400998
* 1.0 kilowatt hour 'Electricity, low voltage {FR}| market for | Cut-off, S - Copied from Ecoinvent' (kilowatt hour, None, None) ==> 0.06176727386616523
* 0.0018000000000000002 ton kilometer 'Transport, freight, lorry 16-32 metric ton, EURO5 {RER}| transport, freight, lorry 16-32 metric ton, EURO5 | Cut-off, S - Copied from Ecoinvent' (ton kilometer, None, None) ==> 0.0002992019577656287
* 0.065 kilogram 'Paper (waste treatment) {GLO}| recycling of paper | Cut-off, S - Copied from Ecoinvent' (kilogram, None, None) ==> 0.0
* 0.01575 kilogram 'Waste paperboard {CH}| t

In [ ]:
import pandas as pd

agb_synthese = pd.read_csv("Agribalyse_Synthese codes ciqual.csv")
ciqual_codes =list(agb_synthese["Code CIQUAL"])

for ciqual_code in ciqual_codes:
    search = agb.search("Ciqual code : " + str(ciqual_code))
    print(search)
    product = search[0]